# Groupby operations

Some imports:

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
try:
    import seaborn
except ImportError:
    pass

pd.options.display.max_rows = 10

## Recap: the groupby operation (split-apply-combine)

The "group by" concept: we want to **apply the same function on subsets of your dataframe, based on some key to split the dataframe in subsets**

This operation is also referred to as the "split-apply-combine" operation, involving the following steps:

* **Splitting** the data into groups based on some criteria
* **Applying** a function to each group independently
* **Combining** the results into a data structure

<img src="img/splitApplyCombine.png">

Similar to SQL `GROUP BY`

The example of the image in pandas syntax:

In [ ]:
df = pd.DataFrame({'key':['A','B','C','A','B','C','A','B','C'],
                   'data': [0, 5, 10, 5, 10, 15, 10, 15, 20]})
df

Using the filtering and reductions operations we have seen in the previous notebooks, we could do something like:


    df[df['key'] == "A"].sum()
    df[df['key'] == "B"].sum()
    ...

But pandas provides the `groupby` method to do this:

In [ ]:
df.groupby('key').aggregate('sum')  # np.sum

In [ ]:
df.groupby('key').sum()

Pandas does not only let you group by a column name. In `df.groupby(grouper)` can be many things:

- Series (or string indicating a column in df)
- function (to be applied on the index)
- dict : groups by values
- levels=[], names of levels in a MultiIndex



In [ ]:
df.groupby(lambda x: x % 2).mean()

## And now applying this on some real data

These exercises are based on the [PyCon tutorial of Brandon Rhodes](https://github.com/brandon-rhodes/pycon-pandas-tutorial/) (so all credit to him!) and the datasets he prepared for that. You can download these data from here: [`titles.csv`](https://drive.google.com/open?id=0B3G70MlBnCgKajNMa1pfSzN6Q3M) and [`cast.csv`](https://drive.google.com/open?id=0B3G70MlBnCgKal9UYTJSR2ZhSW8) and put them in the `/data` folder.

`cast` dataset: different roles played by actors/actresses in films

- title: title of the film
- name: name of the actor/actress
- type: actor/actress
- n: the order of the role (n=1: leading role)

In [ ]:
cast = pd.read_csv('data/cast.csv')
cast.head()

In [ ]:
titles = pd.read_csv('data/titles.csv')
titles.head()

<div class="alert alert-success">
    <b>EXERCISE</b>: Using groupby(), plot the number of films that have been released each decade in the history of cinema.
</div>

In [ ]:
def find_decade(x):
    return int(x / 10)*10

In [ ]:
titles['decade'] = titles['year'].apply(find_decade)

In [ ]:
cast['decade'] = cast['year'].apply(find_decade)

In [ ]:
titles.groupby('decade').count()

<div class="alert alert-success">
    <b>EXERCISE</b>: Use groupby() to plot the number of "Hamlet" films made each decade.
</div>

In [ ]:
# showing also years in which it hasn't been made
titles['title'].eq('Hamlet').astype(int).groupby(titles['decade']).sum()

<div class="alert alert-success">
    <b>EXERCISE</b>: How many leading (n=1) roles were available to actors, and how many to actresses, in each year of the 1950s?
</div>

In [ ]:
cast[(cast['decade'] == 1950) & (cast['n'] == 1)].groupby(['decade','year', 'type']).sum()

<div class="alert alert-success">
    <b>EXERCISE</b>: List the 10 actors/actresses that have the most leading roles (n=1) since the 1990's.
</div>

In [ ]:
cast[(cast['decade'] >= 1990) & (cast['n'] == 1)]['name'].value_counts().iloc[0:10]

<div class="alert alert-success">
    <b>EXERCISE</b>: Use groupby() to determine how many roles are listed for each of The Pink Panther movies.
</div>

In [ ]:
#filling NaNs which must be counted but are not with .count() or .values_count()
    
cast_2 = cast.fillna(0) 

In [ ]:
cast_2[cast_2['title'] == 'The Pink Panther'].groupby('year').count()['n']

<div class="alert alert-success">
    <b>EXERCISE</b>: List, in order by year, each of the films in which Frank Oz has played more than 1 role.
</div>

In [ ]:
cast_3 = cast[cast['name'] == 'Frank Oz'].sort_values(by=['year']).groupby(['year','title']).count()['n']

In [ ]:
cast_3[cast_3 >1]

<div class="alert alert-success">
    <b>EXERCISE</b>: List each of the characters that Frank Oz has portrayed at least twice.
</div>

In [ ]:
cast_4 = cast[cast['name'] == 'Frank Oz']['character'].value_counts()

In [ ]:
cast_4[cast_4 > 1]

## Transforms

Sometimes you don't want to aggregate the groups, but transform the values in each group. This can be achieved with `transform`:

In [ ]:
df

In [ ]:
df.groupby('key').transform('mean')

In [ ]:
def normalize(group):
    return (group - group.mean()) / group.std()

In [ ]:
df.groupby('key').transform(normalize)

In [ ]:
df.groupby('key').transform('sum')

<div class="alert alert-success">
    <b>EXERCISE</b>: Add a column to the `cast` dataframe that indicates the number of roles for the film.
</div>

In [ ]:
#it's late, I don't know what I'm doing, definitely not thinking in pandas terms
n_of_roles = cast.fillna(0).groupby('title').count()['n']  

In [ ]:
#yep, list comprehension overtook the situation: creating serie to concatenate
roles = pd.Series([n_of_roles[name] for name in list(cast['title'])], name='num_roles')    

In [ ]:
cast_5 = cast.join(roles)

In [ ]:
cast_5[cast_5['title']=='Stop Pepper Palmer'] # checking if worked

<div class="alert alert-success">
    <b>EXERCISE</b>: Calculate the ratio of leading actor and actress roles to the total number of leading roles per decade.     
</div>

Tip: you can to do a groupby twice in two steps,  once calculating the numbers, and then the ratios.

In [ ]:
count = cast[cast['n'] == 1][['decade','type','n']].groupby(['decade', 'type']).count()

In [ ]:
total = cast[cast['n'] == 1].groupby(['decade']).sum().drop('year', axis=1)

In [ ]:
count / total

## Intermezzo: string manipulations

Python strings have a lot of useful methods available to manipulate or check the content of the string:

In [ ]:
s = 'Bradwurst'

In [ ]:
s.startswith('B')

In pandas, those methods (together with some additional methods) are also available for string Series through the `.str` accessor:

In [ ]:
s = pd.Series(['Bradwurst', 'Kartoffelsalat', 'Sauerkraut'])

In [ ]:
s.str.startswith('B')

For an overview of all string methods, see: http://pandas.pydata.org/pandas-docs/stable/api.html#string-handling

<div class="alert alert-success">
    <b>EXERCISE</b>: We already plotted the number of 'Hamlet' films released each decade, but not all titles are exactly called 'Hamlet'. Give an overview of the titles that contain 'Hamlet', and that start with 'Hamlet':
</div>

In [ ]:
titles[titles['title'].str.contains('Hamlet')].sort_values(by=['year'])

In [ ]:
titles[titles['title'].str.startswith('Hamlet')].sort_values(by=['year'])

<div class="alert alert-success">
    <b>EXERCISE</b>: List the 10 movie titles with the longest name.
</div>

In [ ]:
indx = titles['title'].str.len().sort_values(ascending=False).index
titles.reindex(indx).iloc[0:10,:]

## Value counts

A useful shortcut to calculate the number of occurences of certain values is `value_counts` (this is somewhat equivalent to `df.groupby(key).size())`)

For example, what are the most occuring movie titles?

In [ ]:
titles.title.value_counts().head()

<div class="alert alert-success">
    <b>EXERCISE</b>: Which years saw the most films released?
</div>

In [ ]:
titles.year.value_counts().head()

<div class="alert alert-success">
    <b>EXERCISE</b>: Plot the number of released films over time
</div>

In [ ]:
graph = titles.year.value_counts().sort_index()

In [ ]:
plt.plot(graph, color='green');
plt.xlabel('Years');
plt.ylabel('Number of Movies');
plt.title('Movies over time');

<div class="alert alert-success">
    <b>EXERCISE</b>: Plot the number of "Hamlet" films made each decade.
</div>

In [ ]:
graph_1 = titles['title'].eq('Hamlet').astype(int).groupby(titles['decade']).sum()

In [ ]:
plt.bar([n for n in range(len(graph_1.index))], graph_1.values, tick_label=graph_1.index, color='green');
plt.xlabel('Years');
plt.ylabel('Number of Movies');
plt.title('Movies over time');
plt.xticks(rotation='vertical');

<div class="alert alert-success">
    <b>EXERCISE</b>: What are the 11 most common character names in movie history?
</div>

In [ ]:
cast.character.value_counts().iloc[0:11]

<div class="alert alert-success">
    <b>EXERCISE</b>: Which actors or actresses appeared in the most movies in the year 2010?
</div>

In [ ]:
cast[cast['year'] == 2010].name.value_counts().iloc[0:10]

<div class="alert alert-success">
    <b>EXERCISE</b>: Plot how many roles Brad Pitt has played in each year of his career.
</div>

In [ ]:
# Brad Pitt started out in 1987.
graph_2 = cast['name'].eq('Brad Pitt').astype(int).groupby(cast['year']).sum().loc[1987:2021]

In [ ]:
plt.figure(figsize=(20,4))
plt.bar([n for n in range(len(graph_2.index))], graph_2.values, tick_label=graph_2.index, color='green');
plt.xlabel('Years');
plt.ylabel('Number of Movies');
plt.title('Brad Pitt\'s movies over time');
plt.xticks(rotation='vertical');


<div class="alert alert-success">
    <b>EXERCISE</b>: What are the 10 most film titles roles that start with the word "The Life"?
</div>

In [ ]:
# I really can't get what it is asking for ._. titles? roles?

<div class="alert alert-success">
    <b>EXERCISE</b>: How many leading (n=1) roles were available to actors, and how many to actresses, in the 1950s? And in 2000s?
</div>

In [ ]:
cast[(cast['n'] == 1) & (cast['decade'] == 1950)][['type','n']].groupby(['type']).count()

In [ ]:
cast[(cast['n'] == 1) & (cast['decade'] == 2000)][['type','n']].groupby(['type']).count()